In [169]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import lightgbm as lgb

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [170]:
df = pd.read_csv("../data/훈련데이터셋/3_훈련데이터셋_LAG_휴일포함.csv")
#df.info()

In [171]:
df.head()

,날짜,전체코드,최저기온(°C),0.5m 지중온도(°C),평균 증기압(hPa),가조시간(hr),평균 상대습도(%),파워,lag_1d,lag_7d,휴일여부
0,2022-12-04,1111010100,-5.2,7.5,1.8,9.7,38.9,146294.6135,NaN,NaN,True
1,2022-12-05,1111010100,-7.0,7.0,2.0,9.7,41.9,175633.8270,146294.6135,NaN,False
2,2022-12-06,1111010100,-3.9,6.6,4.1,9.7,62.8,156084.1910,175633.8270,NaN,False
3,2022-12-07,1111010100,NaN,NaN,NaN,NaN,NaN,NaN,156084.1910,NaN,False
4,2022-12-08,1111010100,0.1,6.6,4.4,9.7,57.5,177018.8420,NaN,NaN,False


In [172]:
df["휴일여부"] = df["휴일여부"].astype(int)

In [173]:
# df["temp_lag_1d"] = (
#     df.groupby("전체코드")["최저기온(°C)"].shift(1)
# )

In [174]:
# 3일간 평균 최저기온
df["temp_3d_mean"] = (
    df.groupby("전체코드")["최저기온(°C)"]
      .shift(1).rolling(3).mean()
)

In [175]:
# 기온 변동성
df["temp_7d_std"] = (
    df.groupby("전체코드")["최저기온(°C)"]
      .shift(1)
      .rolling(7)
      .std()
)

In [176]:
# 단기 변동성
df["temp_3d_std"] = (
    df.groupby("전체코드")["최저기온(°C)"]
      .shift(1)
      .rolling(3)
      .std()
)

In [177]:
# # 전력 가속도
# df["power_accel"] = (
#     df.groupby("전체코드")["lag_1d"]
#       .diff(1)
# )

In [178]:
feature_cols = [
    # 지역
    "전체코드",

    # 날씨
    # "temp_lag_1d",
    "temp_7d_std",
    "temp_3d_std",
    "temp_3d_mean",
    "최저기온(°C)", "0.5m 지중온도(°C)", "평균 증기압(hPa)","가조시간(hr)","평균 상대습도(%)",
    
    # 기간
    "lag_1d", "lag_7d"

]
target_col = "파워"

In [179]:
feature_cols = feature_cols + ["휴일여부"]

In [180]:
weather_cols = [
    "최저기온(°C)",
    "0.5m 지중온도(°C)",
    "평균 증기압(hPa)",
    "가조시간(hr)",
    "평균 상대습도(%)"
]

In [181]:
# new_df = df.dropna(subset=["파워"]).reset_index(drop=True)

In [182]:
new_df = df[["날짜"] + feature_cols + [target_col]].dropna().reset_index(drop=True)

In [183]:
print(len(df), len(new_df))
new_df.isna().sum().sort_values(ascending=False)

233208 131002


날짜               0
전체코드             0
temp_7d_std      0
temp_3d_std      0
temp_3d_mean     0
최저기온(°C)         0
0.5m 지중온도(°C)    0
평균 증기압(hPa)      0
가조시간(hr)         0
평균 상대습도(%)       0
lag_1d           0
lag_7d           0
휴일여부             0
파워               0
dtype: int64

In [184]:
# =========================
# 1) 시간순 Train/Valid/Test 분할
# =========================
new_df = new_df.sort_values("날짜").reset_index(drop=True)

n = len(new_df)
test_size = int(n * 0.20)
valid_size = int(n * 0.10)

train_end = n - (valid_size + test_size)
valid_end = n - test_size

train_df = new_df.iloc[:train_end]
valid_df = new_df.iloc[train_end:valid_end]
test_df  = new_df.iloc[valid_end:]

In [185]:
# =========================
# 2) X / y 분리
# =========================
drop_cols = [target_col, "날짜"]  # 날짜는 직접 입력 피처로 쓰지 않음(우린 파생변수로 이미 반영)
X_train = train_df.drop(columns=drop_cols)
y_train = train_df[target_col]

X_valid = valid_df.drop(columns=drop_cols)
y_valid = valid_df[target_col]

X_test = test_df.drop(columns=drop_cols)
y_test = test_df[target_col]

In [186]:
# =========================
# 4) Xgboost 모델
# =========================
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

xgb_model = XGBRegressor(
    n_estimators=2500,        # 트리 수 ↑ (early stopping 전제)
    learning_rate=0.02,       # 더 천천히 학습

    max_depth=4,              # 깊이 ↓ (과적합 억제)
    min_child_weight=8,      # ★ 핵심: 노이즈 분기 방지

    subsample=0.8,
    colsample_bytree=0.8,

    gamma=0.2,                # 분기 비용 ↑
    reg_alpha=0.5,            # L1 추가 (feature 선택 효과)
    reg_lambda=8.0,          # L2 강화

    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1,
    tree_method="hist"
)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"[XGB] RMSE: {rmse:,.2f}")
print(f"[XGB] R2  : {r2:.4f}")

[XGB] RMSE: 74,571.15
[XGB] R2  : 0.8755


In [187]:
# 중요도 전체

import pandas as pd

imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": xgb_model.feature_importances_
}).sort_values("importance", ascending=False)

imp.head(20)

,feature,importance
9,lag_1d,0.460046
10,lag_7d,0.163855
0,전체코드,0.137391
7,가조시간(hr),0.060195
5,0.5m 지중온도(°C),0.028898
8,평균 상대습도(%),0.027541
3,temp_3d_mean,0.022899
11,휴일여부,0.020662
4,최저기온(°C),0.019777
6,평균 증기압(hPa),0.019772


In [188]:
imp[imp["feature"] == "전체코드"]

,feature,importance
0,전체코드,0.137391


In [189]:
imp[imp["feature"] == "휴일여부"]

,feature,importance
11,휴일여부,0.020662


In [190]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 예측
y_pred = xgb_model.predict(X_test)

# 지표 계산
mse  = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"MSE : {mse:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"MAE : {mae:,.2f}")
print(f"R2  : {r2:.4f}")

MSE : 5,560,855,856.02
RMSE: 74,571.15
MAE : 12,368.57
R2  : 0.8755


In [194]:
import joblib

# 저장
joblib.dump(xgb_model, "xgb_model.pkl")

['xgb_model.pkl']

In [192]:
loaded_model = joblib.load("xgb_model.pkl")

In [193]:
valid_df["전체코드"].nunique()
test_df["전체코드"].nunique()

466